This notebook is about

# How we can censor our texts in dialogue system
We detect the following topics (for russian language):
* Суицид;
* Насилие
* Секс
* ЛГБТ
* Наркотики
* Терроризм
* Политика

The train and dev data contains our own marked up dialogues (by hands) - 1 and marked up OpenSubTitles and LentaRU by matching of obscene words corpus. Test data contain other utterances marked up by our hands -2 (Note, that our data 1 and data 2 has very different styles and distributions if you want).

PS. For more detailes about data write me dirrectly to email.

In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## Test MultiBERT
### PoolingLinearClassifier

### Train

In [2]:
from modules.data import bert_data_clf

In [3]:
train_df_path = "/home/ubuntu/censor/train.csv"
valid_df_path = "/home/ubuntu/censor/dev.csv"
test_df_path = "/home/ubuntu/censor/test.csv"

In [4]:
data = bert_data_clf.LearnDataClass.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2cls_path="/home/ubuntu/censor/idx2cls.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.classifiers import BERTBaseClassifier

In [6]:
model = BERTBaseClassifier.create(len(data.train_ds.cls2idx))

In [7]:
from modules.train.train_clf import NerLearner

In [8]:
num_epochs = 100

In [9]:
data.train_ds.cls2idx

{'1': 0, '0': 1}

In [10]:
learner = NerLearner(
    model, data, "/home/ubuntu/censor/cls.cpt", t_total=num_epochs * len(data.train_dl))

In [11]:
model.get_n_trainable_params()

885903

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [30]:
learner.load_model()

In [18]:
from modules.data.bert_data_clf import get_data_loader_for_predict

In [25]:
dl, ds = get_data_loader_for_predict(data, df_path=test_df_path)

In [31]:
preds = learner.predict(dl)

In [32]:
from sklearn_crfsuite.metrics import flat_classification_report

In [33]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [35]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.9804    0.9903    0.9853      2471
           1     0.8644    0.7574    0.8074       202

   micro avg     0.9727    0.9727    0.9727      2673
   macro avg     0.9224    0.8739    0.8963      2673
weighted avg     0.9716    0.9727    0.9719      2673



### BERTBiLSTMAttnClassifier

In [2]:
from modules.data import bert_data_clf

In [3]:
train_df_path = "/home/ubuntu/censor/train.csv"
valid_df_path = "/home/ubuntu/censor/dev.csv"
test_df_path = "/home/ubuntu/censor/test.csv"

In [4]:
data = bert_data_clf.LearnDataClass.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2cls_path="/home/ubuntu/censor/idx2cls.txt",
    clear_cache=False
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.classifiers import BERTBiLSTMAttnClassifier

In [6]:
model = BERTBiLSTMAttnClassifier.create(len(data.train_ds.cls2idx))

In [7]:
from modules.train.train_clf import NerLearner

In [8]:
num_epochs = 100

In [9]:
data.train_ds.cls2idx

{'1': 0, '0': 1}

In [10]:
learner = NerLearner(
    model, data, "/home/ubuntu/censor/cls.cpt2", t_total=num_epochs * len(data.train_dl))

In [11]:
model.get_n_trainable_params()

2889999

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [13]:
learner.load_model()

In [14]:
from modules.data.bert_data_clf import get_data_loader_for_predict

In [15]:
dl, ds = get_data_loader_for_predict(data, df_path=test_df_path)

In [16]:
preds = learner.predict(dl)

In [17]:
from sklearn_crfsuite.metrics import flat_classification_report

In [18]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [19]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.9899    0.9923    0.9911      2471
           1     0.9031    0.8762    0.8894       202

   micro avg     0.9835    0.9835    0.9835      2673
   macro avg     0.9465    0.9343    0.9403      2673
weighted avg     0.9833    0.9835    0.9834      2673



### BERTBiLSTMAttnClassifier (add open subtitles)

In [39]:
from modules.data import bert_data_clf

In [62]:
train_df_path = "/home/ubuntu/censor/train.csv"
valid_df_path = "/home/ubuntu/censor/dev.csv"
test_df_path = "/home/ubuntu/censor/test2.csv"

In [60]:
data = bert_data_clf.LearnDataClass.create(
    train_df_path=train_df_path,
    valid_df_path=None,
    idx2cls_path="/home/ubuntu/censor/idx2cls.txt",
    clear_cache=True
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [42]:
from modules.models.classifiers import BERTBiLSTMAttnClassifier

In [43]:
model = BERTBiLSTMAttnClassifier.create(len(data.train_ds.cls2idx))

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpa2vfzwcs
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pool

In [44]:
from modules.train.train_clf import NerLearner

In [45]:
num_epochs = 100

In [46]:
data.train_ds.cls2idx

{'0': 0, '1': 1}

In [61]:
learner = NerLearner(
    model, data, "/home/ubuntu/censor/cls.cpt3", t_total=num_epochs * 1)

In [48]:
model.get_n_trainable_params()

2889999

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [49]:
learner.load_model()

In [50]:
from modules.data.bert_data_clf import get_data_loader_for_predict

#### on valid

In [63]:
dl, ds = get_data_loader_for_predict(data, df_path=valid_df_path)

In [ ]:
preds = learner.predict(dl)

In [35]:
from sklearn_crfsuite.metrics import flat_classification_report

In [36]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [37]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.9960    0.9903    0.9931      1243
           1     0.9941    0.9975    0.9958      2031

   micro avg     0.9948    0.9948    0.9948      3274
   macro avg     0.9950    0.9939    0.9945      3274
weighted avg     0.9948    0.9948    0.9948      3274



#### on test

In [51]:
dl, ds = get_data_loader_for_predict(data, df_path=test_df_path)

In [52]:
preds = learner.predict(dl)

In [53]:
from sklearn_crfsuite.metrics import flat_classification_report

In [54]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [55]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.5550    0.4436    0.4931       523
           1     0.5366    0.6444    0.5856       523

   micro avg     0.5440    0.5440    0.5440      1046
   macro avg     0.5458    0.5440    0.5393      1046
weighted avg     0.5458    0.5440    0.5393      1046



In [56]:
errors = []
for pred, feature in zip(preds, ds):
    if pred != feature.cls:
        errors.append({
            "text": " ".join(feature.tokens[1:-1]),
            "pred": pred,
            "true": feature.cls
        })

In [59]:
learner.predict(df=pd.DataFrame(
    {
        "text": ["Привет, пидор", "Путин гад", "что правда, гандон?"], "cls": ["1", "0", "0"]
}))

['0', '0', '0']

### BERTBiLSTMAttnClassifier (add open subtitles and lenta)

In [65]:
from modules.data import bert_data_clf

In [66]:
train_df_path = "/home/ubuntu/censor/train_f.csv"
valid_df_path = "/home/ubuntu/censor/dev_f.csv"
test_df_path = "/home/ubuntu/censor/test2.csv"

In [68]:
data = bert_data_clf.LearnDataClass.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2cls_path="/home/ubuntu/censor/idx2cls.txt",
    clear_cache=True,
    batch_size=64
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [69]:
from modules.models.classifiers import BERTBiLSTMAttnClassifier

In [70]:
model = BERTBiLSTMAttnClassifier.create(len(data.train_ds.cls2idx))

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpd6umgh5k
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pool

In [71]:
from modules.train.train_clf import NerLearner

In [72]:
num_epochs = 100

In [73]:
data.train_ds.cls2idx

{'1': 0, '0': 1}

In [74]:
learner = NerLearner(
    model, data, "/home/ubuntu/censor/cls_f.cpt", t_total=num_epochs * len(data.train_dl))

In [75]:
model.get_n_trainable_params()

2889999

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [77]:
learner.load_model()

In [78]:
from modules.data.bert_data_clf import get_data_loader_for_predict

#### on valid

In [79]:
dl, ds = get_data_loader_for_predict(data, df_path=valid_df_path)

In [80]:
preds = learner.predict(dl)

In [81]:
from sklearn_crfsuite.metrics import flat_classification_report

In [82]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [83]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.9952    0.9894    0.9923      5486
           1     0.9927    0.9967    0.9947      7950

   micro avg     0.9937    0.9937    0.9937     13436
   macro avg     0.9940    0.9931    0.9935     13436
weighted avg     0.9938    0.9937    0.9937     13436



In [84]:
errors = []
for pred, feature in zip(preds, ds):
    if pred != feature.cls:
        errors.append({
            "text": " ".join(feature.tokens[1:-1]),
            "pred": pred,
            "true": feature.cls
        })

In [117]:
learner.predict(df=pd.DataFrame(
    {
        "text": ['дурацкий день', "Путин гад".lower(), "что правда, гандон?"], "cls": ["0", "0", "0"]
}))

['1', '1', '1']

#### on test

In [105]:
dl, ds = get_data_loader_for_predict(data, df_path=test_df_path)

In [106]:
preds = learner.predict(dl)

In [107]:
from sklearn_crfsuite.metrics import flat_classification_report

In [108]:
cls_report = flat_classification_report([f.cls for f in ds], preds, digits=4)

In [109]:
print(cls_report)

              precision    recall  f1-score   support

           0     0.9757    0.9962    0.9858       523
           1     0.9961    0.9751    0.9855       523

   micro avg     0.9857    0.9857    0.9857      1046
   macro avg     0.9859    0.9857    0.9857      1046
weighted avg     0.9859    0.9857    0.9857      1046



In [110]:
errors = []
for pred, feature in zip(preds, ds):
    if pred != feature.cls:
        errors.append({
            "text": " ".join(feature.tokens[1:-1]),
            "pred": pred,
            "true": feature.cls
        })

In [111]:
1 - len(errors)/len(preds)

0.98565965583174

In [112]:
errors

[{'text': 'есть чё?', 'pred': '0', 'true': '1'},
 {'text': 'я квир', 'pred': '0', 'true': '1'},
 {'text': 'тебя возбуждают другие коты?', 'pred': '0', 'true': '1'},
 {'text': 'что ты принимаешь?', 'pred': '0', 'true': '1'},
 {'text': 'ты что куришь?', 'pred': '0', 'true': '1'},
 {'text': 'посоветуй что-нибудь для расширения сознания',
  'pred': '0',
  'true': '1'},
 {'text': 'я колюсь', 'pred': '0', 'true': '1'},
 {'text': 'Чей Крым?', 'pred': '0', 'true': '1'},
 {'text': 'ты голосуешь?', 'pred': '0', 'true': '1'},
 {'text': 'Гнать дуру', 'pred': '0', 'true': '1'},
 {'text': 'АУЕ ворам свободу', 'pred': '0', 'true': '1'},
 {'text': 'Смолить чернушку', 'pred': '0', 'true': '1'},
 {'text': 'майн кампф', 'pred': '0', 'true': '1'},
 {'text': 'По одной из версий, он мог умереть от наркотического опьянения.',
  'pred': '1',
  'true': '0'},
 {'text': 'На одном из них было написано, что за убийство черного кота игрока ненавидит весь мир, а на другом поступок Луиша Фигу был назван "позором "Инт